# Solutions III: Modelling

In [ ]:
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

## Load Data

In [ ]:
# Read in the banking data set.
df = pd.read_csv("../../0_data/banking/bank-additional-full.csv", sep=";")
df.head(3)

In [ ]:
# Split features and labels.
X = df.drop(columns=["duration", "pdays", "y"])
y = df["y"]

## Data Preparation

In [ ]:
# Get column names per data type.
categorical = X.select_dtypes("object").columns

In [ ]:
# Create a OneHotEncoder and StandardScaler.
ohe = OneHotEncoder(sparse_output=False)

In [ ]:
# Set up the ColumnTransformer.
transformer = ColumnTransformer(
    transformers=[
        ("encode_categorical", ohe, categorical),
    ],
    remainder="passthrough",
)

In [ ]:
# Transform the data.
# Note: Use transform="pandas" to get a DataFrame.
transformer.set_output(transform="pandas")
Xt = transformer.fit_transform(X)

In [ ]:
# Quick inspection of the transformed data.
Xt.head(3)

## RandomForest Model

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Create random forest model instance.
rf = RandomForestClassifier(n_jobs=-1)

In [ ]:
# Fit to the data.
rf.fit(Xt, y)

In [ ]:
# Create predictions.
predicted = rf.predict(Xt)

In [ ]:
# Accuracy is extremely high.
(y == predicted).mean()

### Feature Importances

In [ ]:
# Important features; not very readable...
rf.feature_importances_

In [ ]:
# Combine with feature names into a DataFrame
importance = pd.DataFrame({
    "feature": rf.feature_names_in_,
    "importance": rf.feature_importances_,
})

In [ ]:
# Show top 10 features
(
    importance
    .sort_values("importance", ascending=False)
    .round(3)
    .head(10)
)

### Probabilities

In [ ]:
# Get the prediction probabilities
probabilities = rf.predict_proba(Xt)
probabilities[0]

In [ ]:
# First probability correspondes to "no" label.
predicted[0]

In [ ]:
# Combine the data with predictions and probabilities (for "yes").
analysis = (
    df.assign(
        predicted=predicted,
        probability_yes=probabilities[:, 1],
    )
    
)

In [ ]:
# Uncertain predictions are close to the decision boundary.
# Note: Compute the absolute distance to the 0.5 boundary.
analysis = analysis.assign(
    certainty=(0.5 - analysis["probability_yes"]).abs()
)
analysis.sample(3)

In [ ]:
# Find the top 10 most uncertain predictions.
analysis.sort_values("certainty").head(10).T

In [ ]:
# Find the 10 most confident predictions that were wrong.
(
    analysis
    
    # Filter wrong predictions only.
    .query("y != predicted")

    # Sort values on descencing certainty.
    .sort_values("certainty", ascending=False)
    
    .head(10)
    .T
)

## Combine with Pipeline

In [ ]:
# Get column names per data type.
categorical = X.select_dtypes("object").columns
numerical = X.select_dtypes("number").columns

In [ ]:
# Set up the ColumnTransformer.
transformer = ColumnTransformer(
    transformers=[
        ("encode_categorical", OneHotEncoder(), categorical),
    ],
    remainder="passthrough",
)

In [ ]:
# Set up the model.
random_forest = RandomForestClassifier(n_jobs=-1)

In [ ]:
# Combine preparation and model using a Pipeline.
pipe = Pipeline(
    steps=[
        ("preparation", transformer),
        ("model", random_forest),
    ]
)

In [ ]:
# Fit now prepares the data and then feeds it to the model.
pipe.fit(X, y)

In [ ]:
# Predict also prepares the data before feeding it to the model.
pipe.predict(X)

In [ ]:
# Accessing a Pipeline step by its name.
pipe["model"]

In [ ]:
# Getting feature names produced in preparation.
pipe["preparation"].get_feature_names_out()

In [ ]:
# Getting feature importances
pipe["model"].feature_importances_